## Deprecated in favor of MulticlassLogisticDigits.ipyng

In [12]:
# Libraries
import numpy as np
rng =np.random.default_rng()
np.set_printoptions(suppress=True, precision=4)
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from scipy.special import logsumexp
from bokeh.plotting import figure, output_notebook, show
output_notebook()
from tqdm import tqdm
from scipy.special import log_softmax, softmax

Loading BokehJS ...

## Multiclass Regression

### Simulate data

We will simulate some tiny image data, where each image has is an array of five 0-255 values and there
are 4 different images:
x0x0x
00xxx
xxx00
x000x

In [95]:
samples=[60,40,80,120]
variability=40

x0=[30,200,30,200,30]
x1=[200,200,30,30,30]
x2=[30,30,30,200,200]
x3=[30,200,200,200,30]

data = np.concatenate([rng.normal(loc=x,scale=variability,size=(n,5)).astype(int) for n,x in zip(samples,[x0,x1,x2,x3])])
data = np.concatenate([data,np.ones((data.shape[0],1))],axis=1)
# scale the data
data = data/np.max(data,axis=0)

labels = np.array([0]*samples[0]+[1]*samples[1]+[2]*samples[2]+[3]*samples[3])

# rearrange the rows
rows = list(range(sum(samples)))
shuffled = rng.shuffle(rows)
data = data[rows,:]
labels=labels[rows]


In [96]:
labels

array([0, 2, 2, 3, 3, 3, 3, 2, 2, 1, 3, 1, 2, 1, 3, 1, 2, 2, 3, 3, 2, 3,
       3, 3, 0, 1, 2, 3, 2, 1, 3, 1, 3, 3, 2, 3, 2, 3, 3, 0, 2, 2, 0, 2,
       1, 2, 0, 2, 0, 2, 0, 0, 2, 3, 0, 1, 0, 0, 3, 2, 2, 1, 1, 0, 2, 0,
       2, 3, 2, 0, 2, 1, 3, 0, 2, 2, 3, 3, 3, 3, 2, 3, 3, 2, 3, 3, 3, 2,
       0, 0, 3, 2, 2, 2, 3, 1, 2, 3, 2, 2, 0, 2, 3, 3, 0, 3, 3, 3, 1, 0,
       2, 1, 1, 2, 0, 1, 3, 2, 0, 2, 1, 3, 3, 2, 3, 3, 3, 3, 3, 1, 3, 2,
       3, 0, 0, 1, 2, 2, 2, 0, 3, 3, 3, 3, 1, 0, 3, 2, 2, 2, 3, 3, 2, 2,
       3, 3, 2, 3, 3, 2, 1, 3, 0, 3, 3, 3, 2, 0, 3, 0, 2, 0, 3, 3, 0, 1,
       1, 3, 3, 1, 2, 0, 3, 2, 3, 3, 3, 3, 3, 3, 0, 3, 2, 0, 2, 2, 3, 0,
       3, 3, 0, 0, 3, 3, 0, 0, 0, 3, 0, 3, 3, 1, 3, 0, 3, 1, 2, 3, 1, 3,
       2, 0, 3, 2, 3, 1, 2, 1, 2, 1, 2, 3, 2, 2, 1, 3, 3, 0, 3, 3, 3, 3,
       2, 2, 3, 0, 2, 0, 3, 3, 0, 0, 3, 1, 2, 0, 0, 0, 2, 0, 3, 2, 0, 3,
       1, 3, 3, 3, 0, 1, 0, 3, 1, 3, 0, 2, 1, 3, 0, 0, 3, 3, 2, 0, 3, 0,
       2, 2, 3, 2, 3, 3, 3, 3, 1, 3, 1, 3, 1, 2])

In [97]:
### Test the sklearn routines

L=LogisticRegression(solver='liblinear')

#### add the constant feature
#data = (data-data.mean(axis=0))/data.std(axis=0)
L.fit(data,labels)
print("The coefficient matrix")
print(L.coef_)
print("The intercepts")
print(L.intercept_)


The coefficient matrix
[[-1.9397  2.2307 -4.7519  2.1462 -2.8922 -0.7527]
 [ 3.9483  0.701  -1.5145 -3.8929 -0.7742 -0.5721]
 [-0.5478 -4.4847 -1.5197  0.6837  4.413  -0.2815]
 [-1.7003  1.0493  6.3839  0.2173 -1.9019 -1.3504]]
The intercepts
[-0.7527 -0.5721 -0.2815 -1.3504]


In [98]:
print("Accuracy on the test data is {} percent".format(100*L.score(data,labels)))

Accuracy on the test data is 96.33333333333334 percent


In [99]:
# Use sklearn to create a "one-hot" encoding of the labels

E=OneHotEncoder()
Y=E.fit_transform(labels.reshape(-1,1)).toarray()

def descent(x,y, max_iter=1000,nu=.001,M=None):
    """does gradient ascent to maximum likelihood for data=x and labels (one-hot)=y"""
    features = x.shape[1]
    classes = y.shape[1]
    grads=[]
    if M is None:
        M = rng.normal(loc=0,scale=1,size=(features,classes))
    for i in tqdm(range(max_iter)):
        P=softmax(x@M,axis=1)
        grad = data.transpose() @ (Y-P)
        M = M+nu*grad
        grads.append(np.max(np.abs(grad)))
    return M,grads

In [106]:
M,g=descent(data,Y,max_iter=10000,nu=.0001)
M

100%|██████████| 10000/10000 [00:00<00:00, 15607.66it/s]


array([[-2.0992,  4.0066, -1.4879, -1.9407],
       [ 2.6125,  1.4463, -4.5775,  1.3599],
       [-3.8995, -1.0462, -1.6255,  6.749 ],
       [ 2.803 , -3.7401,  1.3572,  0.784 ],
       [-0.677 ,  0.8342,  5.8847, -0.0712],
       [-0.5311, -0.4893,  0.4719, -2.0849]])

In [108]:
### the predicted label is the one with maximum probability
predictions=np.argmax(softmax(data @ M),axis=1)

### accuracy compares predicted to true
correct=((predictions==labels).sum())/predictions.shape[0]

print("Accuracy here is {} percent".format(100* correct))

Accuracy here is 97.66666666666667 percent


In [109]:
f=figure(title='Size of largest coefficient in gradient vs. number of iterations')
f.line(x=list(range(len(g[200:]))),y=g[200:])
show(f)